# "MAXIMUM BETWEENNESS", 
The "MAXIMUM BETWEENNESS" problem is an NP-hard problem that I will tackle in this project using raw optimization techniques.

Problem Statement:

We have a set A and a collection C of ordered triples (a, b, c) consisting of distinct elements from A. The goal is to find a unique function f that maps elements of A to a range from 1 to the size of A, in such a way that the number of triples satisfying a specific order constraint (f(a) < f(b) < f(c) or f(c) < f(b) < f(a)) is maximized.

Moving forward, I will develop optimization methods such as genetic algorithms, simulated annealing, or local search. (It is important to ensure that my methods can handle large datasets and produce high-quality results).


In [2]:
import random
import math
from itertools import permutations

# Brute Force
The brute-force algorithm should provide a comparative analysis of optimization methods.
It should be capable of efficiently handling small instances of the problem and producing optimal solutions to verify the correctness of the optimization approaches.

In [15]:
def brute_force(A, C):
    max_betweenness = 0
    permutations_f = permutations(A)

    for perm in permutations_f:
        f = dict(zip(A, perm))

        current_betweenness = 0
        for (a, b, c) in C:
            if (f[a] < f[b] < f[c]) or (f[c] < f[b] < f[a]):
                current_betweenness += 1
                
        if(current_betweenness > max_betweenness):
            max_betweenness = current_betweenness
            max_f = f.values()

    
    return max_betweenness, max_f

# BNB

In [3]:
def brute_force_BNB(A, C):
    max_betweenness = 0
    C_size = len(C)

    permutations_f = permutations(A)
    for perm in permutations_f:
        f = dict(zip(A, perm))

        current_betweenness = 0
        i=0
        for (a, b, c) in C:
            i += 1
            if (f[a] < f[b] < f[c]) or (f[c] < f[b] < f[a]):
                current_betweenness += 1
            ostatak = C_size - i  
            if (current_betweenness + ostatak <= max_betweenness):
                    break

        if(current_betweenness > max_betweenness):
            max_betweenness = current_betweenness 
            max_f = f.values()
    
    return max_betweenness, max_f

### Helper functions:

In [4]:
# pomocne funkcije koje cemo koristiti u narednim algoritmima

def count_triplets(solution, C):
    count = 0
    for (a, b, c) in C:
        index_a = solution.index(a)
        index_b = solution.index(b)
        index_c = solution.index(c)
        if (index_a < index_b < index_c) or (index_c < index_b < index_a):
            count += 1
    return count

def initial_solution(A):
    return random.sample(A, len(A))


# Local Search

In [5]:
def neighborhood(solution):
    for i in range(len(solution)):
        for j in range(i+1, len(solution)):
            neighbor = solution[:]   #kopija
            neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
            yield neighbor

def local_search_first_improvement(A, C, max_iterations=1000):
    current_solution = initial_solution(A)
    current_score = count_triplets(current_solution, C)

    for _ in range(max_iterations):
        past_solution = current_solution
        for neighbor in neighborhood(current_solution):
            neighbor_score = count_triplets(neighbor, C)
            if neighbor_score > current_score:
                current_solution = neighbor
                current_score = neighbor_score
                break  # cim naletimo na prvo bolje resenje
        if current_score == len(C):
            break
        if past_solution is current_solution:
            break
    
    return current_solution, current_score

def local_search_best_improvement(A, C, max_iterations=1000):
    current_solution = initial_solution(A)
    current_score = count_triplets(current_solution, C)
    best_solution = current_solution[:]
    best_score = current_score
    
    for _ in range(max_iterations):
        for neighbor in neighborhood(current_solution):
            neighbor_score = count_triplets(neighbor, C)
            if neighbor_score > best_score:
                best_solution = neighbor
                best_score = neighbor_score

        if(current_solution == best_solution):
            break;
        current_solution = best_solution
        current_score = best_score
        
    return current_solution, current_score

# Simultated Annealing

Simulated Annealing is a metaheuristic technique that simulates the metal annealing process. This technique does not require an exhaustive search of the entire search space but instead iteratively improves the current solution. Simulated annealing can be useful for large and complex search spaces.

In [5]:
def generate_neighbor(solution):
    neighbor = solution[:]
    i, j = random.sample(range(len(solution)), 2)
    neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
    return neighbor

In [7]:
def simulated_annealing(A, C, initial_temperature=100, cooling_rate=0.95, stopping_temperature=1e-6, max_iterations=1000):
    current_solution = initial_solution(A)
    current_score = count_triplets(current_solution, C)
    best_solution = current_solution
    best_score = current_score
    temperature = initial_temperature
    
    for iteration in range(max_iterations):
        if temperature < stopping_temperature:
            break
        
        neighbor = generate_neighbor(current_solution)
        neighbor_score = count_triplets(neighbor, C)
        
        delta_score = neighbor_score - current_score

                                                    # Metropolisov kriterijum
        if delta_score > 0 or random.uniform(0, 1) < math.exp(delta_score / temperature):
            current_solution = neighbor
            current_score = neighbor_score
        
        if current_score > best_score:
            best_solution = current_solution
            best_score = current_score
        
        temperature *= cooling_rate
    
    return best_solution, best_score

When the temperature is high, the Metropolis criterion is larger, meaning there is a higher probability of accepting a worse solution. As the temperature decreases over time, the probability of accepting worse solutions also decreases, allowing the algorithm to gradually focus on improving the solution quality.

In [8]:
# Primeri strategija hlađenja temperature
def geometric_cooling(initial_temperature, iteration, cooling_rate):
    return initial_temperature * cooling_rate ** iteration

def sqrt_cooling(initial_temperature, iteration, cooling_rate):
    return initial_temperature * cooling_rate / math.sqrt(iteration + 1)

def log_cooling(initial_temperature, iteration, cooling_rate):
    return initial_temperature * cooling_rate / math.log(iteration + 2)

def simulated_annealing_cooling_strategy(A, C, cooling_strategy, initial_temperature=100, cooling_rate = 0.95, stopping_temperature=1e-6, max_iterations=1000):
    current_solution = initial_solution(A)
    current_score = count_triplets(current_solution, C)
    best_solution = current_solution
    best_score = current_score
    temperature = initial_temperature
    
    for iteration in range(max_iterations):
        if temperature < stopping_temperature:
            break
        
        neighbor = generate_neighbor(current_solution)
        neighbor_score = count_triplets(neighbor, C)
        
        delta_score = neighbor_score - current_score

        if delta_score > 0 or random.uniform(0, 1) < math.exp(delta_score / temperature):
            current_solution = neighbor
            current_score = neighbor_score
        
        if current_score > best_score:
            best_solution = current_solution
            best_score = current_score
        
        temperature = cooling_strategy(initial_temperature, iteration, cooling_rate)
    
    return best_solution, best_score

In [17]:
def adaptive_cooling(initial_temperature, iteration, acceptance_rate, target_acceptance_rate, cooling_rate):
    if acceptance_rate > target_acceptance_rate:
        return initial_temperature * cooling_rate
    else:
        return initial_temperature / math.log(iteration + 2)

def simulated_annealing_adaptive(A, C, initial_temperature=100, stopping_temperature=1e-6, max_iterations=1000, target_acceptance_rate=0.5, cooling_rate = 0.95):
    current_solution = initial_solution(A)
    current_score = count_triplets(current_solution, C)
    best_solution = current_solution
    best_score = current_score
    temperature = initial_temperature
    acceptance_count = 0
    total_count = 0
    
    for iteration in range(max_iterations):
        if temperature < stopping_temperature:
            break
        
        neighbor = generate_neighbor(current_solution)
        neighbor_score = count_triplets(neighbor, C)
        
        delta_score = neighbor_score - current_score

        if delta_score > 0 or random.uniform(0, 1) < math.exp(delta_score / temperature):
            current_solution = neighbor
            current_score = neighbor_score
            acceptance_count += 1
        
        total_count += 1
        acceptance_rate = acceptance_count / total_count
        temperature = adaptive_cooling(initial_temperature, iteration, acceptance_rate, target_acceptance_rate, cooling_rate)
    
    return best_solution, best_score


The "adaptive_cooling" function decreases the temperature if the acceptance rate of new solutions is higher than the target acceptance rate; otherwise, it slightly increases it.

In [4]:
def generate_neighbor_with_heuristic(C,solution,num_neighbors_to_generate):
    best_neighbor = None
    best_neighbor_score = -1
    for _ in range(num_neighbors_to_generate):
        neighbor = generate_neighbor(solution)
        neighbor_score = count_triplets(neighbor, C)
        if neighbor_score > best_neighbor_score:
            best_neighbor = neighbor
            best_neighbor_score = neighbor_score
    return best_neighbor

def simulated_annealing_with_intensive_search(A, C, initial_temperature=100, cooling_rate=0.95, stopping_temperature=1e-6, max_iterations=1000, num_neighbors_to_generate=10):
    current_solution = initial_solution(A)
    current_score = count_triplets(current_solution, C)
    best_solution = current_solution
    best_score = current_score
    temperature = initial_temperature
    
    for iteration in range(max_iterations):
        if temperature < stopping_temperature:
            break
        
        neighbor = generate_neighbor_with_heuristic(C ,current_solution, num_neighbors_to_generate)
        neighbor_score = count_triplets(neighbor, C)
        
        delta_score = neighbor_score - current_score

        if delta_score > 0 or random.uniform(0, 1) < math.exp(delta_score / temperature):
            current_solution = neighbor
            current_score = neighbor_score
        
        if current_score > best_score:
            best_solution = current_solution
            best_score = current_score
        
        temperature *= cooling_rate
    
    return best_solution, best_score


# Genetic Algorithm

A Genetic Algorithm is inspired by the process of evolution in nature. It uses a population of solutions and applies genetic operators such as selection, crossover, and mutation to find an optimal solution. This technique is particularly useful for large search spaces and complex optimization problems.

Solution Representation:
Each solution in the search space is represented as an individual or a unit.

Selection:
Choosing parents from the population of solutions. This is usually based on the objective function, where solutions with better fitness (higher objective function values) are more likely to be selected for reproduction.

Crossover:
Creating new individuals by combining the genetic information of selected parents. This process simulates the crossover of genetic material in nature.

Mutation:
Introducing random changes in individuals to maintain population diversity and prevent stagnation in local optima.

Evaluation:
Assessing each individual in the population based on the objective function.

Iteration of the Process:
Repeating selection, crossover, and mutation across generations until a predefined stopping criterion is met.

In [10]:
def initial_population(A, population_size):
    return [random.sample(A, len(A)) for _ in range(population_size)]

def crossover(parent1, parent2):
    crossover_point = random.randint(1, len(parent1) - 1)
    child1 = parent1[:crossover_point] + [gene for gene in parent2 if gene not in parent1[:crossover_point]]
    child2 = parent2[:crossover_point] + [gene for gene in parent1 if gene not in parent2[:crossover_point]]
    return child1, child2

def mutation(solution, mutation_rate):
    if random.random() < mutation_rate:
        i, j = random.sample(range(len(solution)), 2)
        solution[i], solution[j] = solution[j], solution[i]
    return solution

def genetic_algorithm(A, C, population_size=100, generations=1000, crossover_rate=0.8, mutation_rate=0.2):
    population = initial_population(A, population_size)
    
    for generation in range(generations):
        scores = [count_triplets(solution, C) for solution in population]
        
        selected_parents = random.choices(population, weights=scores, k=population_size)
        
        new_population = []
        for i in range(0, population_size, 2):
            if random.random() < crossover_rate:
                child1, child2 = crossover(selected_parents[i], selected_parents[i+1])
                new_population.extend([child1, child2])
            else:
                new_population.extend([selected_parents[i], selected_parents[i+1]])
        
        mutated_population = [mutation(solution, mutation_rate) for solution in new_population]
        
        population = mutated_population

    scores = [count_triplets(solution, C) for solution in population]
    
    best_solution_index = scores.index(max(scores))
    best_solution = population[best_solution_index]
    best_score = scores[best_solution_index]
    
    return best_solution, best_score